In [3]:
import pandas as pd

In [4]:
data = pd.read_csv('aadhar_enrolment_final.csv')

In [4]:
data.columns

Index(['date', 'state', 'district', 'pincode', 'age_0_5', 'age_5_17',
       'age_18_greater'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006029 entries, 0 to 1006028
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   date            1006029 non-null  object
 1   state           1006029 non-null  object
 2   district        1006029 non-null  object
 3   pincode         1006029 non-null  int64 
 4   age_0_5         1006029 non-null  int64 
 5   age_5_17        1006029 non-null  int64 
 6   age_18_greater  1006029 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 53.7+ MB


In [11]:
data.shape

(1006029, 7)

In [14]:
data.duplicated().value_counts()

False    983072
True      22957
Name: count, dtype: int64

In [24]:
data.duplicated(
    subset=[
        'date','state','district','pincode',
        'age_0_5','age_5_17','age_18_greater'
    ]
).value_counts()

False    983072
True      22957
Name: count, dtype: int64

In [19]:
(data['state']=="100000").sum()

np.int64(22)

# District Report

In [15]:
import re

district_view = (
    data[['state', 'district']]
    .dropna()
    .assign(
        district_norm=lambda df:
            df['district']
            .str.lower()
            .str.strip()
            .str.replace(r'[()\-\s]+', ' ', regex=True)
    .str.replace(r'[^a-z0-9 ]', '', regex=True)
    )
)


In [16]:
variation_summary = (
    district_view
    .groupby(['state', 'district_norm'])['district']
    .nunique()
    .reset_index(name='no_of_variations')
    .query('no_of_variations > 1')
)


In [17]:
district_variations = (
    district_view
    .merge(
        variation_summary[['state', 'district_norm']],
        on=['state', 'district_norm']
    )
    .groupby(['state', 'district_norm'])['district']
    .unique()
    .reset_index(name='district_spellings')
)


In [18]:
district_variations

,state,district_norm,district_spellings
0,Bihar,aurangabad bh,"[Aurangabad(bh), Aurangabad(BH)]"
1,Chhattisgarh,janjgir champa,"[Janjgir-champa, Janjgir - Champa, Janjgir Cha..."
2,Jammu and Kashmir,punch,"[Punch, punch]"
3,Jharkhand,seraikela kharsawan,"[Seraikela-Kharsawan, Seraikela-kharsawan]"
4,Karnataka,yadgir,"[Yadgir, yadgir]"
5,Odisha,angul,"[Angul, ANGUL]"
6,Odisha,anugul,"[Anugul, ANUGUL]"
7,Odisha,jajpur,"[JAJPUR, jajpur, Jajpur]"
8,Odisha,nuapada,"[Nuapada, NUAPADA]"
9,Punjab,sas nagar mohali,"[SAS Nagar (Mohali), S.A.S Nagar(Mohali)]"


In [4]:
district_view = (
    data[['state','district']]
    .drop_duplicates()
    .assign(
        norm_space=lambda df:
            df['district']
            .str.lower()
            .str.replace(r'[()\-\s]+', '', regex=True)
            .str.replace(r'[^a-z0-9]', '', regex=True)
    )
)


In [5]:
(
    district_view
    .groupby(['state','norm_space'])['district']
    .unique()
    .reset_index(name='district_spellings')
    .query('district_spellings.str.len() > 1')
)


,state,norm_space,district_spellings
23,Andhra Pradesh,karimnagar,"[Karimnagar, Karim Nagar]"
27,Andhra Pradesh,kvrangareddy,"[K.V.Rangareddy, K.v. Rangareddy]"
28,Andhra Pradesh,mahabubnagar,"[Mahabub Nagar, Mahabubnagar]"
117,Bihar,aurangabadbh,"[Aurangabad(bh), Aurangabad(BH)]"
177,Chhattisgarh,janjgirchampa,"[Janjgir-champa, Janjgir - Champa, Janjgir Cha..."
215,Delhi,northeast,"[North East, North East *]"
231,Gujarat,banaskantha,"[Banas Kantha, Banaskantha]"
251,Gujarat,panchmahals,"[Panchmahals, Panch Mahals]"
255,Gujarat,sabarkantha,"[Sabarkantha, Sabar Kantha]"
257,Gujarat,surendranagar,"[Surendra Nagar, Surendranagar]"


# Pincode

In [5]:
data['pincode_str'] = data['pincode'].astype(str)

pincode_format_issues = data[
    (~data['pincode_str'].str.match(r'^\d{6}$')) |
    (data['pincode'].isna())
]

pincode_format_issues[['state', 'district', 'pincode']]


,state,district,pincode


In [6]:
pincode_district_conflict = (
    data.groupby('pincode')['district']
        .nunique()
        .reset_index(name='district_count')
)

problem_pincodes = pincode_district_conflict[
    pincode_district_conflict['district_count'] > 1
]

problem_pincodes



,pincode,district_count
1,110001,2
3,110003,3
7,110007,2
11,110011,2
16,110016,2
...,...,...
19451,855101,3
19453,855105,2
19455,855107,3
19459,855115,2
